## NSMC(Naver Sentiment Movie Corpis
네이버 영화 리뷰 말뭉치<br>
from Korpora import Korpora


 * 사용하는 모델: beomi/kcbert-base
 *  모델 구조: BertForSequenceClassification (BERT 모델을 감성 분석용 분류기로 변환)
 * 사전 학습된 데이터: 한국어 데이터(Korean)로 사전 학습된 KcBERT 모델
 * 추가 학습하는 데이터: NSMC(네이버 영화 리뷰 감성 분석) 데이터셋

In [1]:
import torch
print("PyTorch Version:", torch.__version__)
print("MPS 지원 여부:", torch.backends.mps.is_available())  # True여야 정상 작동
print("MPS 사용 가능:", torch.backends.mps.is_built())  # True여야 정상 작동

PyTorch Version: 2.6.0
MPS 지원 여부: True
MPS 사용 가능: True


In [2]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print("사용할 디바이스:", device)

사용할 디바이스: mps


### Tokenizer
토큰화 수행 프로그램<br>
kcbert-base 모델

# Pytorch's Data Loader

* 파이토치로 딥러닝 모델을 만들려면 반드시 정의해야 한다.
* 데이터를 배치(batch)단위로 모델에 밀어 넣어주는 역할
* 전체 데이터 가운데 일부 인스턴스를 뽑아 배치를 구성
* 데이터셋은 데이터 로더의 구성 요소 중 하나
* 데이터셋은 여러 인스턴스를 보유

데이터 로더 > 데이터셋 > 인스턴스

* batch는 그 모양이 고정적이어야 할 때가 많다. -> 문장들의 토큰(input_ids) 개수가 같아야 한다.

그래서 batch의 shape을 동일하게 만들어 주는 과정을 collate라고 한다.

### Collate
* list -> pytorch의 tensor로 변환
* batch size 통일

### Pytorch Lightning
https://minjoo-happy-blog.tistory.com/140

In [1]:
import torch
import torch.nn.functional as F
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report, accuracy_score
from Korpora import Korpora

# KcBERT 모델 및 토크나이저 로드
MODEL_NAME = "beomi/kcbert-base"
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

# GPU(MPS) 또는 CPU 설정
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)
model.eval()  # 평가 모드

# NSMC 데이터 다운로드 및 로드
data_dir = "./data/nsmc"
test_file = f"{data_dir}/ratings_test.txt"

# 데이터가 없으면 자동 다운로드
if not os.path.exists(test_file):
    Korpora.fetch("nsmc", root_dir=data_dir)

# NSMC 테스트 데이터 불러오기
test_df = pd.read_csv(test_file, sep="\t").dropna()

# NSMC 데이터셋을 PyTorch Dataset 형식으로 변환
class NsmcDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=128):
        self.encodings = tokenizer(
            df["document"].astype(str).tolist(),
            padding="max_length",
            truncation=True,
            max_length=max_length,
            return_tensors="pt"
        )
        self.labels = torch.tensor(df["label"].tolist(), dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "label": self.labels[idx],
        }

# 데이터셋 및 DataLoader 생성
test_dataset = NsmcDataset(test_df, tokenizer)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 모델 평가 함수 정의
def evaluate_model(model, dataloader):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # 성능 평가 결과 출력
    acc = accuracy_score(all_labels, all_preds)
    print(f"📊 정확도 (Accuracy): {acc * 100:.2f}%")
    print("📊 상세 평가 결과:")
    print(classification_report(all_labels, all_preds, target_names=["부정 😡", "긍정 😊"]))

# 모델 성능 평가 실행
print("🔍 KcBERT 모델 성능 평가 중...")
evaluate_model(model, test_dataloader)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🔍 KcBERT 모델 성능 평가 중...
📊 정확도 (Accuracy): 49.43%
📊 상세 평가 결과:
              precision    recall  f1-score   support

        부정 😡       0.49      0.59      0.54     24826
        긍정 😊       0.50      0.40      0.44     25171

    accuracy                           0.49     49997
   macro avg       0.49      0.49      0.49     49997
weighted avg       0.49      0.49      0.49     49997



In [4]:
from dataclasses import dataclass  
from transformers import BertConfig, BertForSequenceClassification, BertTokenizer, AdamW
import pytorch_lightning as pl

@dataclass
class ClassificationTrainArguments:
    pretrained_model_name: str
    downstream_corpus_name: str
    downstream_corpus_root_dir: str
    downstream_model_dir: str
    learning_rate: float
    batch_size: int

args = ClassificationTrainArguments(
    pretrained_model_name="beomi/kcbert-base",  # KC-BERT 사용
    downstream_corpus_name="nsmc",
    downstream_corpus_root_dir="./data",
    downstream_model_dir="./model",
    learning_rate=5e-5,
    batch_size=32
)

# MPS(GPU) 자동 감지 및 설정
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"사용 디바이스: {device}")

# NSMC 데이터 다운로드
data_dir = f"{args.downstream_corpus_root_dir}/{args.downstream_corpus_name}"
train_file = f"{data_dir}/ratings_train.txt"
test_file = f"{data_dir}/ratings_test.txt"

if not os.path.exists(train_file):
    Korpora.fetch(args.downstream_corpus_name, root_dir=data_dir)

# 모델 및 토크나이저 로드 (순서 수정)
tokenizer = BertTokenizer.from_pretrained(args.pretrained_model_name, do_lower_case=False)

pretrained_model_config = BertConfig.from_pretrained(args.pretrained_model_name, num_labels=2)
model = BertForSequenceClassification.from_pretrained(args.pretrained_model_name, config=pretrained_model_config)

# 모델을 디바이스로 이동
model.to(device)

# 데이터 불러오기 (샘플링 적용)
train_df = pd.read_csv(train_file, sep="\t").dropna().sample(5000)  # 일부 샘플 사용
test_df = pd.read_csv(test_file, sep="\t").dropna().sample(1000)

# NSMC 데이터셋 클래스
class NsmcDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=128):
        self.encodings = tokenizer(
            df["document"].astype(str).tolist(),
            padding="max_length",
            truncation=True,
            max_length=max_length,
            return_tensors="pt"
        )
        self.labels = torch.tensor(df["label"].tolist(), dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "label": self.labels[idx],
        }

# 데이터셋 및 DataLoader 생성
train_dataset = NsmcDataset(train_df, tokenizer)
test_dataset = NsmcDataset(test_df, tokenizer)

train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, num_workers=0, pin_memory=True)
test_dataloader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False, num_workers=0)

# 감성 분석 모델 정의 (PyTorch Lightning)
class SentimentClassificationTask(pl.LightningModule):
    def __init__(self, model, learning_rate=5e-5):
        super().__init__()
        self.model = model
        self.learning_rate = learning_rate

    def forward(self, input_ids, attention_mask):
        return self.model(input_ids, attention_mask=attention_mask)

    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["label"]

        outputs = self(input_ids, attention_mask)
        logits = outputs.logits
        loss = F.cross_entropy(logits, labels)

        self.log("train_loss", loss, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=self.learning_rate)

# 모델 학습 실행
task = SentimentClassificationTask(model)

trainer = pl.Trainer(
    max_epochs=50,  
    accelerator="mps" if torch.backends.mps.is_available() else "cpu",
    log_every_n_steps=10,
    callbacks=[pl.callbacks.EarlyStopping(monitor="train_loss", patience=3)]
)
# 모델 성능 평가 함수
def evaluate_model(model, dataloader):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    print(f"모델 정확도: {acc * 100:.2f}%")

# 모델 평가 실행
print("테스트 데이터셋 평가 중...")
evaluate_model(task.model, test_dataloader)

# 예제 문장 감정 분석 함수
def predict_sentiment(model, text):
    model.eval()
    inputs = tokenizer(text, truncation=True, padding="max_length", max_length=128, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model(inputs["input_ids"], attention_mask=inputs["attention_mask"])
        logits = outputs.logits
        probs = F.softmax(logits, dim=1)
        pred_class = torch.argmax(probs, dim=1).item()

    label_map = {0: "부정 😡", 1: "긍정 😊"}
    print(f"입력 문장: {text}")
    print(f"예측 결과: {label_map[pred_class]} (확률: {probs[0][pred_class] * 100:.2f}%)\n")

# 예제 문장 감정 분석 실행
sample_texts = [
    "이 영화는 정말 최고였어!",
    "완전 최악이야, 시간 낭비했어.",
    "그냥 그랬어. 별로 감흥이 없었어.",
    "배우 연기가 너무 훌륭했어!",
    "스토리가 너무 지루했어."
]

for text in sample_texts:
    predict_sentiment(task.model, text)

사용 디바이스: mps


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


테스트 데이터셋 평가 중...
모델 정확도: 51.10%
입력 문장: 이 영화는 정말 최고였어!
예측 결과: 부정 😡 (확률: 51.04%)

입력 문장: 완전 최악이야, 시간 낭비했어.
예측 결과: 부정 😡 (확률: 52.02%)

입력 문장: 그냥 그랬어. 별로 감흥이 없었어.
예측 결과: 부정 😡 (확률: 57.55%)

입력 문장: 배우 연기가 너무 훌륭했어!
예측 결과: 부정 😡 (확률: 52.94%)

입력 문장: 스토리가 너무 지루했어.
예측 결과: 부정 😡 (확률: 52.74%)

